# Lecture 5 Text Generation

This notebook will show you how to generate text content with OpenAI's API for ChatGPT.

Below is the overview of this notebook.

<ol type = 1>
  <li> Generate Text with ChatGPT</li>
  <li> Create Tweets with Topic and Sentiment
  <li> Few-Shot Learning of Tweet Style</li>
  <li> Customer Service Twitter Bot </li>
</ol>

Before starting, select "Runtime->Factory reset runtime" to start with your directories and environment in the base state.

If you want to save changes to the notebook, select "File->Save a copy in Drive" from the top menu in Colab.  This will save the notebook in your Google Drive.


# Clones, Installs, and Imports


## Clone GitHub Repository

In [1]:
!git clone https://github.com/zlisto/social_media_genAI

import os
os.chdir("social_media_genAI/main")

Cloning into 'social_media_genAI'...
remote: Enumerating objects: 1815, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 1815 (delta 65), reused 140 (delta 47), pack-reused 1655 (from 2)
Receiving objects: 100% (1815/1815), 177.06 MiB | 22.51 MiB/s, done.
Resolving deltas: 100% (136/136), done.


## Install Packages

In [2]:
pip install -q -r requirements.txt


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.3/548.3 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu1

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codecs  #this let's us display tweets properly (emojis, etc.)
import textwrap as tr
from tqdm import tqdm  #progress bar for for loops
from IPython.display import HTML
import json
import textwrap
import plotly.express as px
from sklearn.decomposition import PCA
import openai

from scripts.genai import GenAI

#this option makes it so tweets display nicely in a dataframe
pd.set_option("display.max_colwidth", None)

#this code sets the font sizes for plots
plt.rcParams.update({'axes.labelsize': 18,
                     'xtick.labelsize': 14,
                     'ytick.labelsize': 14,
                     'figure.figsize':(8,6),
                     'axes.grid':True})



## OpenAI API Key

You will need to input your OpenAI API key.  

1.  First you need to create an account with OpenAI here: https://auth0.openai.com/u/signup?state=hKFo2SBWS3JUVEdmQmdzZXo5ckhpY3R5NEFlc2NPWWc3WHhvRqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIG9kTDB4LV83aEdnN3pRU3VUYnVZemlnZkFURFo2RDhno2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q

2. Once you have an account, copy your API key from here: https://beta.openai.com/account/api-keys

3. Finally, paste your key below:  `OPENAI_API_KEY = "your OpenAI API key"`

In [4]:
OPENAI_API_KEY = ""


In [5]:
# Initialize GenAI
jarvis = GenAI(OPENAI_API_KEY)

text = "Who are you?"
instructions = "You are a helpful assistant."
response = jarvis.generate_text(text, instructions)
print(f"Human:{text}\nAI:{response}")

Human:Who are you?
AI:I am an AI language model designed to assist with a variety of tasks, such as answering questions, providing information, and helping with problem-solving. How can I assist you today?


# Generate Tweets with ChatGPT





## Instructions and Prompt

We will provide `instructions` to the AI telling it its general function.  We then provide a `prompt` with the specific task we want it to to.  


In [6]:
instructions = '''You are going to help create content for a social
media account.  You will be asked to write clever tweets that will get
engagement.  Return only the text of the tweet.'''

prompt = '''Write a tweet.'''



## Display tweet

We give the `prompt` and `instructions` to `generate_text` to get the output, which in this case is a tweet.  


To display the tweet, use the `display_tweet` function which makes it look like a tweet.  This function was written by ChatGPT after looking at a screenshot of a tweet.

In [7]:
tweet = jarvis.generate_text(prompt, instructions)
screen_name = 'AI_zlisto'
jarvis.display_tweet(tweet, screen_name);

## Getting help

Each function in the GenAI class has some help text, which you can access by typing `help(GenAI.function_name)`.  You can also get the information about the whole `GenAI` class by typing `help(GenAI)`.  Try it below.  The things `help` prints are called doc strings and the AI does a nice job of writing them for you.

In [9]:
help(GenAI.generate_text)

Help on function generate_text in module scripts.genai:

generate_text(self, prompt, instructions='You are a helpful AI named Jarvis', model='gpt-4o-mini', output_type='text')
    Generates a text completion using the OpenAI API.
    
    This function sends a prompt to the OpenAI API with optional instructions to guide the AI's behavior. 
    It supports specifying the model and output format, and returns the generated text response.
    
    Parameters:
    ----------
    prompt : str
        The user input or query that you want the AI to respond to.
    
    instructions : str, optional (default='You are a helpful AI named Jarvis')
        System-level instructions to define the AI's behavior, tone, or style in the response.
    
    model : str, optional (default='gpt-4o-mini')
        The OpenAI model to use for generating the response. You can specify different models like 'gpt-4', 'gpt-3.5-turbo', etc.
    
    output_type : str, optional (default='text')
        The format of 

# Controlling Tweet Content, Sentiment, and Style

## Tweet Topic

We can make a tweet about a topic my modifying the `prompt`.

In [12]:
instructions = '''You are going to help create content for a social
media account.  You will be asked to write clever tweets that will get
engagement.  Return only the text of the tweet.'''

topic = 'healthy eating can cure better than big pharma drugs.'
prompt = f'''Write a tweet about {topic}.'''
screen_name = 'RFKJr_Official'
tweet = jarvis.generate_text(prompt, instructions)
jarvis.display_tweet(tweet, screen_name);

## Tweet Sentiment

We can control the sentiment of the tweet by choosing a value for `sentiment` and modifying the prompt to include the desired sentiment along with the range of the sentiment and what the extreme values mean.  The function `create_sentiment_prompt` takes the sentiment value and sentiment range and creates this part of the prompt. We can then add this to the end of our basic prompt.

In [21]:
def create_sentiment_prompt(sentiment, sentiment_min, sentiment_max):
  prompt = f'''Give the tweet a sentiment of {sentiment}
  where sentiment ={sentiment_min} means very negative and
  sentiment = {sentiment_max} means very postive.'''
  return prompt

In [22]:
instructions = '''You are going to help create content for a social
media account.  You will be asked to write clever tweets that will get
engagement. Return only the text of the tweet.'''

topic = 'healthy eating'
screen_name = 'RFKJr_Official'
sentiment_min = -10
sentiment_max =  10
sentiment = 8

prompt = f'''Write a tweet about {topic}.'''
prompt+= create_sentiment_prompt(sentiment,
                                  sentiment_min,
                                  sentiment_max)
tweet = jarvis.generate_text(prompt, instructions)
print(f"Sentiment:{sentiment}\n")
jarvis.display_tweet(tweet, screen_name);


Sentiment:8



## Tweet Style

We can add a description of the style of the tweet in the `instructions`. To do this, create a string `instructions_style` that tells the AI the style of the content creator, then add this to the base `instructions`.

In [23]:
instructions = '''You are going to help create content for a social
media account.  You will be asked to write clever tweets that will get
engagement. Return only the text of the tweet.'''

instructions_style = '''
Use a serious, persuasive tone with a focus on public health and personal freedom.
Speak with conviction, emphasizing the importance of making America healthy again.
Incorporate strong, evidence-based arguments, and be critical of corporate influence, especially from big pharma.
Use thoughtful, clear language that appeals to both logic and moral responsibility, inspiring trust and a call to action for health and environmental justice.
'''

instructions += instructions_style

topic = 'healthy eating'
screen_name = 'RFKJr_Official'

sentiment_min = -10
sentiment_max =  10
sentiment = 8

prompt = f'''Write a tweet about {topic}.'''
prompt+= create_sentiment_prompt(sentiment,
                                  sentiment_min,
                                  sentiment_max)
tweet = jarvis.generate_text(prompt, instructions)
print(f"Sentiment:{sentiment}\n")
jarvis.display_tweet(tweet, screen_name);

Sentiment:8



# Few-Shot Learning of Tweet Style

We will give ChatGPT the tweets of a Twitter user so it can learn their style.  

To collect the tweets of a Twitter user, you can use the Chrome plug-in TwExportly:
https://chromewebstore.google.com/detail/twexportly-export-tweets/hbibehafoapglhcgfhlpifagloecmhfh?hl=en




## Load Data

We have tweets from a variety of users that we will use to teach the AI tweeting style.  The data is in the file `"data/TwExportly/TwExportly_RobertKennedyJr_tweets_2025_02_02.csv"`. We will load the tweets into a dataframe `df`.




In [6]:
filename = "data/TwExportly/TwExportly_RobertKennedyJr_tweets_2025_02_02.csv"
df = pd.read_csv(filename)
df.text.sample(2)

,text
343,@SethAbramson Is there any Democrat who still believes in the First Amendment?
306,The principal objective of the FDA today is to serve the mercantile interests of pharmaceutical companies. Let's get @realDonaldTrump elected so I can go to Washington and rid our government agencies of corruption. https://t.co/ntNfaHa3Gn


## Few-Shot Learning

We add to the `instructions` a random sample of  tweets from this person.  The more examples you give, the more money it costs.  The functon `create_examples_prompt` will create the prompt with the sampled tweets.  It takes as input a dataframe `df` with the tweets and the number of tweet samples you want `nsamples`.

In [16]:
def create_examples_prompt(df, nsamples=1):
    """
    Generates a prompt to guide AI in mimicking the tweeting style based on example tweets.

    Parameters:
    ----------
    df : DataFrame
        A pandas DataFrame containing a 'text' column with tweet content.
    nsamples : int, optional
        Number of sample tweets to include in the prompt (default is 1).

    Returns:
    -------
    str
        A formatted prompt that instructs the AI to generate tweets matching the provided style.
    """
    # Ensure we don't sample more tweets than available
    tweets = df if nsamples >= len(df) else df.sample(nsamples)

    # Format the sample tweets
    tweets_str = ""
    for _, row in tweets.iterrows():
        text = row.text.replace("\n", " ").strip()
        tweets_str += f'- "{text}"\n'

    # Construct the prompt
    prompt = f"""
You are an AI trained to write tweets that perfectly mimic the style, tone, and voice of the following examples.

**Instructions:**
- Pay attention to the word choice, sentence structure, and emotional tone.
- Match the use of hashtags, emojis, punctuation, and formatting as shown in the examples.
- Capture the rhythm and flow of the tweets, including any humor, sarcasm, or formal tone.
- Dont  use hashtags as this is not cool anymore
**Example Tweets:**
{tweets_str}

Now, write new tweets that sound like they were written by the same person.
"""

    return prompt.strip()



In [17]:
nsamples = 100
instructions = """You are going to help create content for a social
media account.  You will be asked to write clever tweets that will get
engagement."""

#add sampled tweets of screen_name to instructions
instructions +=  create_examples_prompt(df, nsamples)

print(instructions)

You are going to help create content for a social
media account.  You will be asked to write clever tweets that will get
engagement.You are an AI trained to write tweets that perfectly mimic the style, tone, and voice of the following examples.

**Instructions:**
- Pay attention to the word choice, sentence structure, and emotional tone.
- Match the use of hashtags, emojis, punctuation, and formatting as shown in the examples.
- Capture the rhythm and flow of the tweets, including any humor, sarcasm, or formal tone.
- Dont  use hashtags as this is not cool anymore
**Example Tweets:**
- "@realDonaldTrump @DonaldJTrumpJr"
- "Some specifics about how Donald Trump and I intend to Make America Healthy Again! https://t.co/A6rXQwj0qH"
- "You can be a part of this mosaic capturing the iconic MAHA handshake.  Donate $47 today to secure your tile and upload your image to help put Trump back in the White House and me to D.C. to Make America Healthy Again. 🇺🇸  BUY NOW ➡️ https://t.co/1TpU5zzGYf ht

## Tweeting about a topic

We can now use these `instructions` to tweet about a topic in the style of the chosen `screen_name`.  

In [18]:
topic = 'drinking raw milk is good for you.'
screen_name = 'RobertKennedyJr'
ntweets = 3


prompt = f'''Write a tweet about {topic}
in the style of these tweets.
Return only the tweet text.'''


print(f"Tweets of {screen_name}")

for i in range(ntweets):
  tweet = jarvis.generate_text(prompt, instructions)
  print(f"\tTweet {i}")
  jarvis.display_tweet(tweet,screen_name);

Tweets of RobertKennedyJr
	Tweet 0


	Tweet 1


	Tweet 2


# Customer Service Bot

We can make a customer service bot that replies to complaint tweets.

## Load Data

Load the data in `"data/tweet_complaints_att.csv"`.  These are tweets of people complaining about the AT&T mobile phone outage and the small compensation the company gave its customers.

In [19]:
# prompt: Load the data in "data/tweet_complaints_americanairlines.csv" into a dataframedf_complaints`. Print the head of the dataframe.

df_complaints = pd.read_csv('data/tweet_complaints_att.csv',
                            quotechar='"',
                            encoding='ISO-8859-1')
df_complaints.head()


,text
0,Why is \n@ATT\n contacting only some of their customers affected by the outage and not all? And offering a credit wtf ??
1,"@FOX2now\n \n@KMOV\n So \n@ATT\n is only going to credit their customers $5 for being without service during the nationwide outage. This is newsworthy. Absolutely pathetic. My husband missed out on work, $5 isn't going to cover the loss of money he missed out on."
2,@ATTNEWS\n and \n@ATT\nLet it be more than $5 bucks plz!
3,@ATTNEWS\n and \n@ATT\nOh man what am I going to do with me 5 bucks
4,@ATT\nYeah but $5 not gone make this right ..


## Instructions



We will make `instructions` that tells the AI to respond to the complaints by empathizing with the customer, expressing anger that they are inconvenienced, and asks them to be patient while they handle it.  

We will give some `instructions_style` that give the AI a personality.

We have a tweet `tweet_att` from AT&T that they posted to address the issue.  

In [20]:
instructions = '''You are a helpful customer service agent that
will respond to complaints about AT&T on Twitter.
You will try to empathize with the customer, acknolwedging their
concerns directly and also expressing anger that they are
inconvenienced, and ask them to be patient.  The message below
is what upset the customers so much:
'''

instructions_style ='''\nYou will use the tone of a British woman
from the Royal Family who is very sweet and polite.
Keep your tweet within 280 characters.\n
'''


#This is the message AT&T posted on their website that upset people so much
tweet_att = '''\nWe recognize the frustration Thursday’s
outage caused & know we let many of our customers down.
To help make it right, we are applying a credit to potentially
impacted accounts to help reassure our customers of our
commitment to reliably connect them - anytime and anywhere.
It will take 1-2 billing cycles to see the credit, depending
on when their bill closes.  For more info: http://att.com/makeitright\n'''



In [21]:
instructions+=tweet_att
instructions+=instructions_style

print(instructions)

You are a helpful customer service agent that
will respond to complaints about AT&T on Twitter.
You will try to empathize with the customer, acknolwedging their
concerns directly and also expressing anger that they are
inconvenienced, and ask them to be patient.  The message below
is what upset the customers so much:

We recognize the frustration Thursday’s
outage caused & know we let many of our customers down.
To help make it right, we are applying a credit to potentially
impacted accounts to help reassure our customers of our
commitment to reliably connect them - anytime and anywhere.
It will take 1-2 billing cycles to see the credit, depending
on when their bill closes.  For more info: http://att.com/makeitright

You will use the tone of a British woman
from the Royal Family who is very sweet and polite.
Keep your tweet within 280 characters.




## Reply to Complaints

We will go through each complaint tweet in `df_complaints` and have the AI write a response tweet to it.  The `prompt` just tells the AI what the complaint tweet is.  For fun, lets have the AI create a screen name for each user.  We can do this using the `instructions_screenname` string.

In [22]:
instructions_screenname = """Create a random user screen name for
a user who is angry at AT&T for not having service all day.
Return only the screen name."""
for index, row in df_complaints.iloc[0:2].iterrows():
  text = row.text.replace('\n', ' ')
  prompt = f'''The complaint is {text}'''
  screen_name = jarvis.generate_text("",instructions_screenname )

  response = jarvis.generate_text(prompt, instructions )

  print(f"\tComplaint:")
  jarvis.display_tweet(text, screen_name);
  print(f"\tResponse:")
  jarvis.display_tweet(response,'ATT');
  print("-" * 150)


	Complaint:


	Response:


------------------------------------------------------------------------------------------------------------------------------------------------------
	Complaint:


	Response:


------------------------------------------------------------------------------------------------------------------------------------------------------


# Other Text Content

For text content, you can also try

1. Tweet

2. Email

3. Substack article

4. Song lyircs (you can make the song at https://suno.com/)



## Create content function

The function `create_content` takes a `topic` and `content_type` and returns an instance of the content type about the topic.

In [23]:
def create_content(topic, content_type):
  instructions = f'''You will be given a content type and a topic.
  You will create the desired content type about the given topic.
  Return the answer as an easy to read and nicely formatted HTML
  with dark mode background.'''

  prompt = f"Content: {content_type}\nTopic: {topic}"
  content = jarvis.generate_text(prompt, instructions)
  content = content.replace('```html', '').replace('```', '')
  return content






In [33]:
topic = """Drink raw milk"""
content_type = """A substack post written by a medical doctor
Dr. Zaman MD, Harvard Med class of 2011"""

content = create_content(topic, content_type)
display(HTML(content))